In [2]:
%pylab inline
import os
import gc
import glob
import numpy as np
import astropy.io.fits as pyfits
import astropy.table as astTable
import maputil as mapUtil

Populating the interactive namespace from numpy and matplotlib


# Prepare small catalogs

In [ ]:
colnames=['tract','Id','ra','dec','g1','g2','weight','bias_m','dnn_zbest','dnn_zbest_std','demp_zbest','demp_zbest_std','mizuki_zbest','mizuki_zbest_std']
for fieldname in mapUtil.field_names:
    print(fieldname)
    data=pyfits.getdata('datDir/catalog_obs_reGaus_no_m/%s_no_m.fits' %fieldname)
    zata=pyfits.getdata('datDir/catalog_others/%s_pz_2.fits' %fieldname)
    ra=data['i_ra']
    dec=data['i_dec']
    e1      =   data['i_hsmshaperegauss_e1']
    e2      =   data['i_hsmshaperegauss_e2']
    m_b     =   np.zeros(len(data))
    Id      =   data['object_id']
    tract   =   data['tract']
    c1_b    =   data['i_hsmshaperegauss_derived_shear_bias_c1']
    c2_b    =   data['i_hsmshaperegauss_derived_shear_bias_c2']
    weight  =   data['i_hsmshaperegauss_derived_weight']
    dnnzBest=   zata['dnnz_photoz_best']
    dnnzStd =   zata['dnnz_photoz_std_best']
    dempzBest=   zata['demp_photoz_best']
    dempzStd =   zata['demp_photoz_std_best']
    mizukizBest=   zata['mizuki_photoz_best']
    mizukizStd =   zata['mizuki_photoz_std_best']
    print(np.sum(np.isnan(zata['mizuki_photoz_best'])))
    w_A     =   np.sum(weight)
    m_bA    =   np.sum(m_b*weight)/w_A
    Res_A   =   1.-np.sum(data['i_hsmshaperegauss_derived_rms_e']**2.*weight)/w_A
    print('response: %s' %Res_A)
    print('multiplicative bias: %s' %m_bA)
    g1      =   1./(1.+m_bA)*(e1/2./Res_A-c1_b)
    g2      =   1./(1.+m_bA)*(e2/2./Res_A-c2_b)
    dataOut=astTab.Table(names=colnames,data=[tract,Id,ra,dec,g1,g2,weight,m_b,dnnzBest,dnnzStd,dempzBest,dempzStd,mizukizBest,mizukizStd])
    dataOut.meta.update({'ra_min':np.min(ra)})
    dataOut.meta.update({'dec_min':np.min(dec)})
    dataOut.meta.update({'ra_max':np.max(ra)})
    dataOut.meta.update({'dec_max':np.max(dec)})
    dataOut.write('wrkDir/reGaus_catalog/%s.fits' %fieldname,overwrite=True)
    if False:
        break
    else:
        del g1,g2,data,m_b,c1_b,c2_b,dnnzBest,dnnzStd,dempzBest,dempzStd,mizukizBest,mizukizStd,dataOut
        gc.collect()

# Divide into groups

In [33]:
pad=1.
for fieldname in mapUtil.field_names:
    outDir='wrkDir/reGaus_catalog/%s_groups/' %fieldname
    if not os.path.isdir(outDir):
        os.mkdir(outDir)
    
    data=pyfits.getdata('wrkDir/reGaus_catalog/%s.fits' %fieldname)
    if fieldname=='VVDS':
        addR=(data['ra']<300.).astype(int)*360
        data['ra']+=addR
    ymin=np.min(data['dec'])
    ymax=np.max(data['dec'])
    ny=mapUtil.field_ny[fieldname]
    ybound=np.linspace(ymin,ymax,ny+1)
    ylow=ybound[:-1];yhigh=ybound[1:]
    ylow=ylow-pad;yhigh=yhigh+pad
    for iy in range(ny):
        ymean= (ybound[iy]+ybound[iy+1])/2.
        mask0= (data['dec']>ymean+iy*1.-1.)&(data['dec']<=ymean+iy*1.)
        dataT= data[mask0]
        xmin=  np.min(dataT['ra'])
        xmax=  np.max(dataT['ra'])
        del dataT
        gc.collect()
        nx  =  int((xmax-xmin)/6.)+1
        xbound=np.linspace(xmin,xmax,nx+1)
        xlow=xbound[:-1];xhigh=xbound[1:]
        xlow=xlow-pad;xhigh=xhigh+pad
        for ix in range(nx):
            mask=(data['ra']>xlow[ix])&(data['ra']<xhigh[ix]) \
                &(data['dec']>ylow[iy])&(data['dec']<yhigh[iy])
            dataU=data[mask]
            if np.sum(mask)>100. and np.max(dataU['dec'])>ylow[iy]+pad*2. and np.min(dataU['dec'])<yhigh[iy]-pad*2.:
                pyfits.writeto(os.path.join(outDir,'src-group-%d%d.fits' %(iy,ix)),dataU)
        del mask0,dataU
        gc.collect()
    del data
    gc.collect()

# Match Poz

In [ ]:
pnames=['dempz','dnnz','mizuki']
for fieldname in mapUtil.field_names:
    inDir='wrkDir/reGaus_catalog/%s_groups/' %fieldname
    pzDir=os.path.join('datDir/catalog_tracts/','%s_tracts/' %fieldname)
    assert os.path.isdir(inDir)
    fnames=glob.glob(os.path.join(inDir,'src-group-*.fits'))
    for fname in fnames:
        data=astTable.Table.read(fname)
        ids=data['Id',]
        ids.rename_column('Id','ID')
        tracts=list(set(data['tract']))
        print(tracts)
        pzAll=[]
        for tract in tracts:
            key=pnames[0]
            pfname=os.path.join(pzDir,'%s_pz_pdf_%s.fits' %(tract,key))
            pzs=astTable.Table.read(pfname)
            pzAll.append(astTable.join(left=ids,right=pzs,keys='ID',join_type='inner'))
            del pzs,pfname
            gc.collect()
        pzAll=astTable.vstack(pzAll)
        pzAll.sort('ID')
        assert np.all(data['Id']==pzAll['ID'])
        del ids,tracts,data
        gn=fname.split('-')[-1].split('.')[0]
        ofname=os.path.join(inDir,'poz-%s-group-%s.fits' %(key,gn))
        pzAll.write(ofname)
        del pzAll
        gc.collect()

In [ ]:
nsim=100
for fieldname in mapUtil.field_names:
    print(fieldname)
    inDir='wrkDir/reGaus_catalog/%s_groups/' %fieldname
    assert os.path.isdir(inDir)
    fnames=glob.glob(os.path.join(inDir,'src-group-*.fits'))
    for fname in fnames:
        gn=fname.split('-')[-1].split('.')[0]
        outDir='wrkDir/reGaus_catalog/%s_groups/mock-group-%s/' %(fieldname,gn)
        if not os.path.isdir(outDir):
            os.mkdir(outDir)
        else:
            continue
        data=pyfits.getdata(fname)
        nobj=len(data)
        for i in range(nsim):
            catMock   =   data.copy()
            np.random.seed(i)
            phi = 2.0 * np.pi * np.random.rand(nobj)
            g1n,g2n   =   mapUtil.rotCatalog(data['g1'],data['g2'],phi)
            catMock['g1']=g1n
            catMock['g2']=g2n
            ofname=os.path.join(outDir,'sim-%d.fits'%i)
            pyfits.writeto(ofname,catMock,overwrite=True)
            del catMock,phi,g1n,g2n,ofname
            gc.collect()
        del data
        gc.collect()

XMM
GAMA09H
